PENDING 
Use the haarcascade extension to find the face and take several pics of person making up the data set 
Use map to store all of this data and use it to recognise and print the name when the same face is encountered again 
Use KNN method for face matching 

In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
id=1
dict={}

In [2]:
def NewFace(id,dict):
    name=input("Enter your name")
    directory="./recogdata/"+name
    cap=cv2.VideoCapture(0)
    haarcascade=cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
    skip=0
    faces_data=[]
    while True :
        ret , frame = cap.read()
        
        if ret==False :
            continue
        
        gray_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        faces= haarcascade.detectMultiScale(gray_frame,1.3,5)
        cv2.imshow("Video Section",frame)
        if len(faces)==0:
            continue
        faces=sorted(faces,key=lambda f:f[2]*f[3])[-1]
        x,y,w,h=faces
        offset=10
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255))
        cv2.imshow("Video Section",frame)
        face_section=frame[y-offset:y+offset+h,x-offset:x+offset+w]
        face=cv2.resize(face_section,(100,100))
        if(skip%10==0):
            faces_data.append(face)
        skip=skip+1
        key_pressed=cv2.waitKey(1) & 0xFF
        if key_pressed==ord('q'):
            break
    faces_data=np.asarray(faces_data)
    faces_data=faces_data.reshape((faces_data.shape[0],-1))
    #print(face_data.shape)
    faces_data=np.concatenate((faces_data,id*np.ones((faces_data.shape[0],1))),axis=1)
    np.save(directory,faces_data)
    print("Data stored successfully at "+directory+".npy")
    cap.release()
    dict[id]=name
    id=id+1
    cv2.destroyAllWindows()
    return id,dict

In [9]:
id,dict=NewFace(id,dict)

Data stored successfully at ./recogdata/sak.npy


In [4]:
id , dict

(2, {1: 'saksham'})

In [5]:
def distance(x,x_):
    return np.sum((x-x_)**2)/len(x)

In [6]:
def kmean(xy,image):
    pred=[]
    n=xy.shape[1]
    for i in range(len(xy)):
        pred.append((distance(xy[i][0:n-1],image),xy[i][-1]))
    pred=sorted(pred,key=lambda x:x[0])[0:4]
    print(pred)
    dict={}
    ans=-1
    for i,j in pred:
        ans=j
        if j in dict:
            dict[j]+=1
        else:
            dict[j]=1
    print(dict)
    for i in dict.items():
        if dict[i[0]]>dict[ans]:
            ans=i[0]
    return ans

In [7]:
def facerecog(dict):
    cap=cv2.VideoCapture(0)
    while True :
        ret,frame=cap.read()
        if ret==False:
            continue
        face_cascade=cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
        gray_scale=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        faces=face_cascade.detectMultiScale(gray_scale,1.3,5)
        if len(faces)==0:
            continue
        faces=sorted(faces,key=lambda f:f[2]*f[3])[-1]
        x,y,w,h=faces
        offset=10
        face_section=frame[y-offset:y+offset+h,x-offset:x+offset+w]
        face=cv2.resize(face_section,(100,100))
        face=np.asarray(face).flatten()
        xy=np.load('./recogdata/'+dict[1]+'.npy')
        for i in range(1,len(dict)):
            temp=np.load('./recogdata/'+dict[i+1]+'.npy')
            print(temp.shape)
            xy=np.concatenate((xy,temp))
        print(xy)
        id=kmean(xy,face)
        name=dict[id]
        print(name)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255))
        cv2.imshow('frame',frame)
        key_pressed=cv2.waitKey(1) & 0xFF
        if key_pressed==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()  

In [10]:
facerecog(dict)

(9, 30001)
[[ 81. 106. 142. ... 124. 131.   1.]
 [103. 128. 154. ... 132. 142.   1.]
 [144. 158. 176. ... 155. 163.   1.]
 ...
 [150. 164. 180. ... 159. 166.   2.]
 [152. 168. 180. ... 157. 160.   2.]
 [148. 160. 178. ... 161. 167.   2.]]
[(520.0446333333333, 2.0), (550.4929, 2.0), (558.9917333333333, 2.0), (590.6761333333334, 2.0)]
{2.0: 4}
sak
(9, 30001)
[[ 81. 106. 142. ... 124. 131.   1.]
 [103. 128. 154. ... 132. 142.   1.]
 [144. 158. 176. ... 155. 163.   1.]
 ...
 [150. 164. 180. ... 159. 166.   2.]
 [152. 168. 180. ... 157. 160.   2.]
 [148. 160. 178. ... 161. 167.   2.]]
[(392.9929666666667, 2.0), (404.0536, 2.0), (405.00713333333334, 2.0), (418.76023333333336, 2.0)]
{2.0: 4}
sak
(9, 30001)
[[ 81. 106. 142. ... 124. 131.   1.]
 [103. 128. 154. ... 132. 142.   1.]
 [144. 158. 176. ... 155. 163.   1.]
 ...
 [150. 164. 180. ... 159. 166.   2.]
 [152. 168. 180. ... 157. 160.   2.]
 [148. 160. 178. ... 161. 167.   2.]]
[(392.9929666666667, 2.0), (404.0536, 2.0), (405.00713333333334

In [9]:
abcd={1:1,2:2,3:3}
print(abcd.items())

dict_items([(1, 1), (2, 2), (3, 3)])
